
# 0.0 Import

In [2]:
!pip install pandas
import pandas as pd
df=pd.read_csv('Datasets/kc_house_data.csv')

     |████████████████████████████████| 9.7 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 15.4 MB 10.6 MB/s eta 0:00:01    |██████▋                         | 3.2 MB 16.2 MB/s eta 0:00:01
^C
ERROR: Operation cancelled by user


# 1.0 Qual a quantidade de imoveis por nivel?


In [3]:
##nivel 0: preço entre 0 e 321,950 $
##nivel 1: preço entre 321,950 $ e 450,000$
##nivel 2: preço entre 450,000$ e 645,000 $
##nivel 3: preço acima de 645,000 

In [4]:
df['level']='None'
df.loc[(df['price']>=0)&(df['price']<321950),'level']='level_0'
df.loc[(df['price']>=321950)&(df['price']<450000),'level']='level_1'
df.loc[(df['price']>=450000)&(df['price']<645000),'level']='level_2'
df.loc[df['price']>=645000,'level']='level_3'


## Loop for

In [ ]:
for i in range(len(df)):
    if (df.loc[i,'price']>=0)&(df.loc[i, 'price']<321950):
        df.loc[i, 'nivel']='level_0'
    elif (df.loc[i,'price']>=321950)&(df.loc[i, 'price']<450000):
        df.loc[i, 'nivel']='level_1'
    elif (df.loc[i,'price']>=450000)&(df.loc[i, 'price']<645000):
        df.loc[i, 'nivel']='level_2'
    else:
        df.loc[i, 'nivel']='level_3'

## While 


In [5]:
import requests as r
url='https://jobs.github.com/positions.json?description=python&page=1'
reponse=r.request('GET',url)

##Json

In [6]:
reponse.json()[0]

{'id': '798a5097-75bf-4659-90fd-78caf35999a1',
 'type': 'Full Time',
 'url': 'https://jobs.github.com/positions/798a5097-75bf-4659-90fd-78caf35999a1',
 'created_at': 'Thu Feb 11 15:07:30 UTC 2021',
 'company': 'Rijksoverheid',
 'company_url': 'http://www.werkenvoornederland.nl',
 'location': 'Den Haag',
 'title': 'Clusterbeheerder / systeembeheerder',
 'description': '<p><strong>Clusterbeheerder / systeembeheerder</strong>\n<strong>Ministerie van Justitie en Veiligheid, Nederlands Forensisch Instituut</strong></p>\n<p><strong>Functieomschrijving</strong>\nAls clusterbeheerder ga je aan de slag bij team Crypto, dat verantwoordelijk is voor het ontcijferen van digitale bewijsstukken. Jij wordt verantwoordelijk voor het beheer en onderhoud van het rekencluster bij het Nederlands Forensisch Instituut (NFI). De taken en verantwoordelijkheden die bij deze functie horen, zijn het beheer en onderhoud van de rekennodes, het volgen van updates in de gebruikte software, en het testen en in-stalle

##Trasformare em data frame

In [7]:
df1=pd.DataFrame(reponse.json()[0], index=[0])
df1[['id','type']]

,id,type
0,798a5097-75bf-4659-90fd-78caf35999a1,Full Time


##While

In [8]:
import requests as r
dataset=pd.DataFrame()
i=1
while True:
    print('page:{}'.format(i))
    url='https://jobs.github.com/positions.json?&page={}'.format(i)
    response=r.request('GET',url)
    
    if response.json()!=[]:
        data=response.json()[0]
        df1=pd.DataFrame(data,index=[0])
        dataset=pd.concat([dataset,df1],axis=0)
        i=i+1
    else:
        break


page:1
page:2
page:3
page:4
page:5
page:6
page:7


# 2.0 Adicione informaçoes ao imovel

##Preparaçao

In [ ]:
df.head()

In [ ]:
from geopy.geocoders import Nominatim
geolocator=Nominatim(user_agent='geoapiExercises')
response=geolocator.reverse('47.5112,-122.257')

In [ ]:
response

In [ ]:
response.raw

In [ ]:
response.raw['address']

In [ ]:
print(response.raw['address']['road'])
print(response.raw['address']['neighbourhood'])
print(response.raw['address']['city'])
print(response.raw['address']['county'])
print(response.raw['address']['state'])
print(response.raw['address']['postcode'])
print(response.raw['address']['country'])
print(response.raw['address']['country_code'])

#modelo

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
data=pd.read_csv('Datasets/kc_house_data.csv')

#Create empty rows
data['road']='NA'
data['house_number']='NA'
data['city']='NA'
data['county']='NA'
data['state']='NA'
data['neighbourhood']='NA'

geolocator=Nominatim(user_agent='geoapiExercises')

for i in range(len(data)):
    print('Loop:{}/{}'.format(i,len(data)))
    
    query=str(data.loc[i,'lat'])+','+str(data.loc[i,'long'])

#API Request
    response=geolocator.reverse(query)

#populate data
    if 'house_number' in response.raw['address']:
        data.loc[i,'house_number']=response.raw['address']['house_number']
    if 'road' in response.raw['address']:
        data.loc[i,'road']        =response.raw['address']['road']
    if 'city' in response.raw['address']:
        data.loc[i,'city']        =response.raw['address']['city']
    if 'county' in response.raw['address']:
        data.loc[i,'county']      =response.raw['address']['county']
    if 'state' in response.raw['address']:
        data.loc[i,'state']       =response.raw['address']['state']
    if 'neighbourhood' in response.raw['address']:
        data.loc[i,'neighbourhood']=response.raw['address']['neighbourhood']





In [ ]:
data.head()

# 3.0 Adicione o nivel do imovel no mapa com uma cor

# 4.0 Adicione o preço do imovel como o tamanho do ponto no mapa

In [ ]:
import plotly.express as px
df=pd.read_csv('Datasets/kc_house_data.csv')

houses=df[['id','long', 'lat', 'price']].copy()
for i in range(len(houses)):
    if  houses.loc[i,'price']<=321950:
        houses.loc[i,'level']=0
        
    elif (houses.loc[i,'price']>321950)&(houses.loc[i, 'price']<=450000):
        houses.loc[i, 'level']=1
        
    elif (houses.loc[i,'price']>=450000)&(houses.loc[i, 'price']<=645000):
        houses.loc[i, 'level']=2
        
    else:
        houses.loc[i, 'level']=3
        
houses['level']=houses['level'].astype(int)

fig=px.scatter_mapbox(houses,
                  lat='lat',
                  lon='long',
                  color='level',
                  size='price',
                  color_continuous_scale=px.colors.cyclical.IceFire,
                  size_max=15,
                  zoom=10)

fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(height=300,margin={'r':0,'t':0,'l':0,'b':0})
fig.show()

# 5.0 Adicione opçoes de filtro para eu fazer as minhas proprias analises

In [ ]:
#1.Importar as bibliotecas
import pandas as pd
import plotly.express as px
import ipywidgets as widgets 
from ipywidgets import fixed
df=pd.read_csv('Datasets/kc_house_data.csv')

In [ ]:
#2.Criar as colunas necessarias

df['is_waterfront']=df['waterfront'].apply(lambda x:'yes' 
                                           if x==1 else 'no')

df['level']=df['price'].apply(lambda x:  0 if x<321950 else
                                         1 if (x>321950)&(x<450000)else
                                         2 if (x>450000)&(x<645000)else 3)
df['level']=df['level'].astype(int)
style={'description_width':'initial'}

#3. Criar os botoes interativos
price_limit=widgets.IntSlider(
    value=540000,
    min=75000,
    max=77000000,
    step=1,
    description='Maximun price',
    disable=False,
    style=style
)

waterfront_bar=widgets.Dropdown(
    options=df['is_waterfront'].unique().tolist(),
    value   ='yes',
    description='Water View',
    disable=False,
)

#4. criar as funçoes que farao os filtros

def update_map(df,waterfront, limit):
    houses=df[(df['price']<=limit)&
              (df['is_waterfront']==waterfront)][['id','lat','long','price','level']]

    
#5. Plotar o mapa

    fig=px.scatter_mapbox(houses,
          lat='lat',
          lon='long',
          color='level',
          size='price',
          color_continuous_scale=px.colors.cyclical.IceFire,
          size_max=15,
          zoom=10)

    fig.update_layout(mapbox_style='open-street-map')
    fig.update_layout(height=300,margin={'r':0,'t':0,'l':0,'b':0})
    fig.show()


#5. Escrever a interaçao  
widgets.interactive(update_map, df=fixed(df),waterfront=waterfront_bar, limit=price_limit)

# 6.0 Adicione opçoes de filtro no ultimo dashboard enviado

In [ ]:
import ipywidgets as widgets 
from matplotlib import gridspec
from matplotlib import pyplot as plt

In [ ]:
data=pd.read_csv('Datasets/kc_house_data.csv')

#change data format
data['year']=pd.to_datetime(df['date']).dt.strftime('%Y')
data['date']=pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
data['year_week']=pd.to_datetime(df['date']).dt.strftime('%Y-%U')

#Widgets to control data
date_limit=widgets.SelectionSlider(
    options=data['date'].sort_values().unique().tolist(),
    value='2014-12-01',
    description='Disponivel',
    continuosus_update=False,
    orientation='horizontal',
    readout=True
)

def update_map(df, limit):
    df=data[data['date']>=limit].copy()
    
    fig=plt.figure(figsize=(21,12))
    specs = gridspec.GridSpec(ncols=2, nrows=2, figure=fig)
    
    ax1=fig.add_subplot(specs[0,:]) #first rows
    ax2=fig.add_subplot(specs[1,0]) #second rows   first column
    ax3=fig.add_subplot(specs[1,1]) #second rows   second column
    
    by_year=df[['id','year']].groupby('year').sum().reset_index()
    ax1.bar (by_year['year'], by_year['id'])
    
    by_day=df[['id','date']].groupby('date').mean().reset_index()
    ax2.plot (by_day['date'], by_day['id'])
    ax2.set_title('title:Avg Price by Day')
    
    by_week_of_year=df[['id','year_week']].groupby('year_week').mean().reset_index()
    ax3.bar (by_week_of_year['year_week'], by_week_of_year['id'])
    ax3.set_title('title:Avg Price by week of year')
    plt.xticks(rotation=60);

In [ ]:
widgets.interactive(update_map,df=fixed(df), limit=date_limit)